#### 1. Naver 뉴스 크롤링
- 미세먼지와 관련된 뉴스기사를 크롤링하여, 기사 본문을 training data로 사용
- Naver API 사용
- 시도 이후, 성능이 좋지 못하면 어머니들 커뮤니티에서 '미세먼지' 키워드로 크롤링할 것

In [1]:
import pickle

with open('secret/naver_key.txt', 'rb') as f : 
    data = pickle.load(f)

In [2]:
import urllib.request
import urllib.parse

from bs4 import BeautifulSoup
def get_news_link(search_word, start, display) : 
    URL = 'https://openapi.naver.com/v1/search/news.xml?'
    # API에서 요청하는 모든 변수 URL로 전달
    sort = 'sort=sim'
    start = '&start=' + str(start) # 1 ~ 1000
    display = '&display=' + str(display) # 10 ~ 100
    query = '&query=' +  urllib.parse.quote_plus(str(search_word)) # 검색어를 입력받아 UTF-8로 전환
    URL = URL + sort + start + display + query
    headers = {
        'Host' : 'openapi.naver.com',
        'User-Agent' : 'curl/7.43.0',
        'Accept' : '*/*',
        'Content-Type' : 'application/xml',
        'X-Naver-Client-Id' : data[0],
        'X-Naver-Client-Secret' : data[1],
     }
     # request로 헤더 정보 전달
    req = urllib.request.Request(URL, headers=headers)
    # header정보를 포함한 request 객체를 HTTP로 전달
    f = urllib.request.urlopen(req)
    resultXML = f.read()
    xmlsoup = BeautifulSoup(resultXML, 'html.parser')
    texts = xmlsoup.select("description")
    ls = []
    for text in texts : 
        ls.append(text.text)
    return ls[1 : ]

In [3]:
sentences = get_news_link("미세먼지", 1, 100)
sentences

['[뉴스데스크] ◀ 앵커 ▶ 지금 뒤로 보시는 장면이 <b>미세먼지</b>에 휩싸인 평양의 모습입니다. <b>미세먼지</b>에 있어서 남과 북의 사정이 그리 다르지 않습니다. 판문점 선언 이후 남북이 손잡고 할 일로 <b>미세먼지</b>... ',
 'ㆍ오늘도 <b>미세먼지</b> 나쁨 ㆍ김동환 지음 |휴머니스트 | 248쪽 | 1만5000원 1982년 5월6일자 경향신문엔 ‘마스크는 이제부터 논-디 마스크로!’라는 제목의 광고가 실렸다. ‘<b>미세</b>한 <b>먼지</b>’로부터 자신의 건강을 보호하자는... ',
 '그렇기 때문에 5월은 <b>미세먼지</b>와 자외선 차단에 더욱 신경 써야 하는 시기이다. 그렇다면 외부 유해환경으로부터 피부 보호를 위한 방법에는 어떤 것이 있을까?<b>미세먼지</b>로부터 피부를 보호하는 기능을 갖춘... ',
 '[앵커] 앞으로 <b>미세먼지</b>와 오존을 발생시키는 질소산화물을 배출하는 사업장에 부과금이 매겨집니다. 환경부는 관련 개정안을 오늘부터 한 달여 간 입법예고합니다. 최서희 기자가 보도합니다. [리포트] 공장에서... ',
 '바로 <b>미세먼지</b>다. ‘안티더스트(Anti-dust)’ 혹은 ‘안티 폴루션(Anti-pollution)’은 요즘 소비시장에서 가장 뜨거운 키워드다. 3일 통계청은 2018년 3월 온라인쇼핑 거래액이 사상 최대인 9조 원을 돌파했다고 밝히며, 그... ',
 '서울시민들에게 여쭤보면 <b>미세먼지</b>와 일자리 문제를 해결해달라고 하더라. 7년간 풀지 못했던 삶의 문제, 서울시민들의 문제를 바꾸고 싶다. 행정·교육·일자리·복지 등에 4차 산업혁명을 접목하겠다고 공약했다. 4차... ',
 '[서울신문] 서울 송파구는 <b>미세먼지</b>의 심각성이 사회적 문제로 대두됨에 따라 ‘찾아가는 <b>미세먼지</b> 환경교실’을 운영한다고 3일 밝혔다. 오는 10일 신천동 잠동초 5학년 교실로 찾아가 130명의 초등학생을 대상으로... ',
 '야외 행사도 위축 분위기…주최측 마스

---

### 목적 : 대화형 챗봇 구현. 
1. 사용자의 말을 인식
2. 사용자가 미세먼지 정보를 원하는지, 잡담을 원하는지에 대한 인식
3. 사용자가 미세먼지 정보를 원할 경우
> 3-1 지역 정보를 정확히 인식, 인식 못할 경우 다시 물어보아 오타 혹은 지역명에 대한 확인 요구
4. 사용자가 잡담을 원하는 경우
> 4-1. 이후 기능 구현

#### 가설 1. word2vec의 CBOW
- 2번 단계, 사용자가 정보를 원하는지 혹은 잡담을 원하는지에 대한 구분
- text 파일 사용, 성능 향상을 위한 특수 기호 및 영어의 경우 stemming 사용
- 이후 tokenization 수행, 이 때 tokenization은 string -> list를 의미.  
ex) [['token', 'token'], ['token', 'token']]

In [1]:
import codecs
from konlpy.tag import Twitter
twitter = Twitter()

from gensim.models import Word2Vec
import multiprocessing

C:\Users\ledes\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Codec이란?
- 파이썬에서 일반적으로 유니코드를 다룰 때 사용하는 decode, encode는 대용량 데이터인 경우, 모든 데이터가 메모리에 올라감
- ** codecs ** 모듈을 통해 파일을 조금씩 읽어, 메모리에도 약간의 데이터만 올려줄 수 있음

In [15]:
#서울 주소 정보 가져오기(구, 동)
test = open("datas/address_seoul.txt", 'r')
address_list = []
lines = test.readlines()
for line in lines : 
    line = (line).split("|")
    address_list.append(line[3 : 5 + 1])

In [21]:
#word2vec parameters
parameters = {
    "sg" : 0, # skip-gram
    "size" : 100, # dimesionality of the feature vectors
    "window" : 3,
    "alpha" :0.01, #learning rate
    "batch_words" : 10000, #사전 구축 시 한 번에 몇 개의 단어를 읽을 것인가
    "iter" : 10, # epoch과 유사
    "workers" : multiprocessing.cpu_count(), #cpu
    
}

In [22]:
model = Word2Vec(**parameters)

In [23]:
model.build_vocab(address_list)

In [25]:
model.train(address_list, total_examples = 100, epochs = 100, )

(84085820, 195361500)

In [48]:
model.save('model')

In [42]:
model.wv.most_similar("태평로1가", topn = 3)

[('순화동', 0.9966834187507629),
 ('봉래동2가', 0.996522843837738),
 ('봉래동1가', 0.9962891340255737)]

In [55]:
model.wv.similarity("흑석동", "도봉구")

0.38406586802287679

In [51]:
model.wv.most_similar(positive = ["흑석동", "종로구"], negative = ["상도동"])

[('신월동', 0.44938164949417114),
 ('목동', 0.4383370578289032),
 ('신정동', 0.42917102575302124),
 ('양천구', 0.42728814482688904),
 ('옥인동', 0.4272083640098572),
 ('도봉구', 0.4132745862007141),
 ('누상동', 0.40563732385635376),
 ('시흥동', 0.3965455889701843),
 ('독산동', 0.3907299339771271),
 ('창동', 0.3893088698387146)]

#### Comment : 
 word2vec 실습 결과 깨달은 사실
- word2vec 기본은 center word를 바탕으로 주변 context를 판단하거나(skip-gram), 주변 context를 바탕으로 center word를 판단하는(CBOW) 방식이 있음
- 구현해야 하는 기능은 **오타가 없다는 전제 하에, API가 인식하지 못하는 지역명을 바로 잡아주는 것(예를 들어, 충북, 구로5동 등)**
- 해결 아이디어 : 전국 시,도 등 주소가 적힌 csv파일을 찾아 훈련시켜볼 것

갑자기 생각난 아이디어!  
- 오타가 없다는 전제 하에 API가 인식하지 못하는 지역명 바로잡기
> 전국 주소를 word2vec으로 학습
- 이외 데이터 학습은 seq2seq로

In [ ]:
corpus = codecs.open("corpus.txt", 'w', encoding = 'utf-8')

In [18]:
def flat(content) : 
    return ["{}/{}".format(word, tag) for word, tag in twitter.pos(content)]

In [ ]:
tokenize_sentences = []
for sentence in sentences : 
    tokenize_sentences.append(flat(sentence))

In [ ]:
clr_sentences = []
for sentence in tokenize_sentences :
    for word in sentence : 
        if "Punctuation" not in word and "Foreign" not in word : 
            clr_sentences.append(word)

In [ ]:
corpus.write()

In [ ]:
for sentence in sentences : 
    corpus.write(' '.join(flat(sentence + '\n')))

In [ ]:
class SentenceReader:

    def __init__(self, filepath):
        self.filepath = filepath

    def __iter__(self):
        for line in codecs.open(self.filepath, encoding='utf-8'):
            yield line.split(' ')

In [ ]:
sentence_vocab = SentenceReader("corpus" + time + ".txt")
sentence_train = SentenceReader(("corpus" + time + ".txt"))